Exercise 1

Implement KNN by hand for just 2 dimensions with normalization.

This is easy because:

    funct: You normalize your data in another table
    funct: You code a simple euclid distance function
    funct: You take a point and calculate the distance to all points
    funct: You take the list from above and sort it
    funct: You aggregate by target variable
    funct: you take the max to determine the targe class

you are finished!

Note: This is the only chance to implement a machine learning algorithm by hand and hence learn something from the ground up!

[For SoSe 2022: Using no libs = full points; Using python helper libs -2] 

In [1]:
import pandas as pd
import numpy as np

df=pd.read_csv("/Users/friederikebuchner/Documents/Beuth/Data Science/IBM/KNN_ex_1_example_data.csv")

df

,Movie Title,# of kicks,# of kisses,type of movie
0,California Man,3,104,Romance
1,He's not reall into dudes,2,100,Romance
2,Beautiful Woman,1,81,Romance
3,Kevin Longblade,101,10,Action
4,Robo Slayer 3000,99,5,Action
5,Amped,98,2,Action
6,?,18,90,?


***Normalize Data***

In [2]:
scaled_df = df.copy()
cols_to_norm = ['# of kicks', '# of kisses']

for column in cols_to_norm:
    scaled_df[column]= df[column].apply(lambda x: (x - df[column].min()) / (df[column].max() - df[column].min()))

***Extract Test data into extra DataFrame***

In [3]:
test_row = pd.DataFrame()
test_row = test_row.append(scaled_df.loc[scaled_df['Movie Title'] == '? '])

### append extra row because pandas doesn't let me select fields from single row-Dataframes correctly
### which leads to calculation errors
test_row = test_row.append({'Movie Title':'Testtesttest'}, ignore_index=True)
test_row

/var/folders/jb/b4kcc8352qlcrywkmmp0fxtr0000gp/T/ipykernel_36965/2159849685.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_row = test_row.append(scaled_df.loc[scaled_df['Movie Title'] == '? '])
/var/folders/jb/b4kcc8352qlcrywkmmp0fxtr0000gp/T/ipykernel_36965/2159849685.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_row = test_row.append({'Movie Title':'Testtesttest'}, ignore_index=True)


,Movie Title,# of kicks,# of kisses,type of movie
0,?,0.17,0.862745,?
1,Testtesttest,NaN,NaN,NaN


***Remove Test Row from dataset***

In [4]:
scaled_df = scaled_df.loc[scaled_df['Movie Title'] != '? ']
scaled_df

,Movie Title,# of kicks,# of kisses,type of movie
0,California Man,0.02,1.000000,Romance
1,He's not reall into dudes,0.01,0.960784,Romance
2,Beautiful Woman,0.00,0.774510,Romance
3,Kevin Longblade,1.00,0.078431,Action
4,Robo Slayer 3000,0.98,0.029412,Action
5,Amped,0.97,0.000000,Action


***Find Point that is to be calculated against all the other points***

In [5]:
pointB = pd.Series([test_row['# of kicks'][0], test_row['# of kisses'][0]])
pointB

0    0.170000
1    0.862745
dtype: float64

***Calculate distance from all Points to point in question***

In [6]:
column_names = ['Movie Title', 'distance']
distanceTable = pd.DataFrame(columns = column_names)

for i in range(len(scaled_df)):
    pointA = pd.Series([scaled_df['# of kicks'][i], scaled_df['# of kisses'][i]])
    new_row = {'index': i, 'Movie Title' : scaled_df['Movie Title'][i], 'distance': sum((pointA - pointB)**2)**0.5}
    distanceTable = distanceTable.append(new_row, ignore_index=True)
    
distanceTable

/var/folders/jb/b4kcc8352qlcrywkmmp0fxtr0000gp/T/ipykernel_36965/3092192148.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  distanceTable = distanceTable.append(new_row, ignore_index=True)
/var/folders/jb/b4kcc8352qlcrywkmmp0fxtr0000gp/T/ipykernel_36965/3092192148.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  distanceTable = distanceTable.append(new_row, ignore_index=True)
/var/folders/jb/b4kcc8352qlcrywkmmp0fxtr0000gp/T/ipykernel_36965/3092192148.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  distanceTable = distanceTable.append(new_row, ignore_index=True)
/var/folders/jb/b4kcc8352qlcrywkmmp0fxtr0000gp/T/ipykernel_36965/3092192148.py:7: FutureWarning: The frame.append method is deprecated and will be removed

,Movie Title,distance,index
0,California Man,0.20332,0.0
1,He's not reall into dudes,0.187648,1.0
2,Beautiful Woman,0.191535,2.0
3,Kevin Longblade,1.141949,3.0
4,Robo Slayer 3000,1.162129,4.0
5,Amped,1.176575,5.0


***Sort distance table***

In [7]:
distanceTable = distanceTable.sort_values('distance')
distanceTable

,Movie Title,distance,index
1,He's not reall into dudes,0.187648,1.0
2,Beautiful Woman,0.191535,2.0
0,California Man,0.20332,0.0
3,Kevin Longblade,1.141949,3.0
4,Robo Slayer 3000,1.162129,4.0
5,Amped,1.176575,5.0


***Find k from sqrt(instances) and find k-nearest-indices***

In [8]:
k = int(np.sqrt(len(distanceTable)))
k_nearest_indices = distanceTable.head(k).index
k_nearest_indices

Int64Index([1, 2], dtype='int64')

***Find the corresponding k-nearest-types-of-movie***

In [9]:
decider = df.iloc[k_nearest_indices]
decider['type of movie']

1    Romance 
2    Romance 
Name: type of movie, dtype: object

***Find out if all k-nearest-movie types are similar and if so assign movie type to movie in question***

In [10]:
type_of_movie = decider['type of movie'].head(1)
type_check = bool(True)
for j in decider['type of movie']: 
    print(j)
    type_check = (j == type_of_movie).all()
        
if (type_check):
    ###again: doesn't select field correctly. why?
    print("Movie '?' is of type", type_of_movie)
else:
    print("Movie '?' is of type unknown.")
    

Romance 
Romance 
Movie '?' is of type 1    Romance 
Name: type of movie, dtype: object
